# Gradient Descent – Lernschritte für $W_Q$ (vereinfachtes Diagonal-Beispiel)

Dieses Notebook reproduziert das Rechenbeispiel aus den Folien:

- Eingabevektor (Fokus): $x = [1, 2, 1, 0]$
- Startgewichte: $W_Q = I$ (nur Diagonale betrachtet)
- Modell (vereinfacht):
  $$\hat z = \sum_i w_{ii} \cdot x_i$$
- Zielwert: $z = 0.5$
- Loss:
  $$L = \tfrac12(\hat z - z)^2$$
- Gradient:
  $$\frac{\partial L}{\partial w_{ii}} = (\hat z - z)\,x_i$$
- Update:
  $$w_{ii}^{neu} = w_{ii} - \eta\,\frac{\partial L}{\partial w_{ii}}$$

Hinweis: Das ist didaktisch vereinfacht (nur Diagonale). In echten Modellen ist $W_Q$ voll und $\hat z$ entsteht über Attention, nicht direkt so.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)


## Setup (wie auf der Folie)

In [ ]:
x = np.array([1.0, 2.0, 1.0, 0.0])
z_target = 0.5
eta = 0.1

# Wir betrachten nur die Diagonale von W_Q als Parametervektor w
# w = [w11, w22, w33, w44]
w0 = np.array([1.0, 1.0, 1.0, 1.0])

print("x =", x)
print("z_target =", z_target)
print("eta =", eta)
print("Start w =", w0)


## Hilfsfunktionen

In [ ]:
def forward(w: np.ndarray, x: np.ndarray) -> float:
    """Vereinfachtes Modell: z_hat = sum_i w_ii * x_i"""
    return float(np.sum(w * x))

def loss(z_hat: float, z: float) -> float:
    return 0.5 * (z_hat - z) ** 2

def grad_w(w: np.ndarray, x: np.ndarray, z: float) -> np.ndarray:
    """Gradient der Diagonale: dL/dw_ii = (z_hat - z) * x_i"""
    z_hat = forward(w, x)
    return (z_hat - z) * x

def step(w: np.ndarray, x: np.ndarray, z: float, eta: float) -> np.ndarray:
    return w - eta * grad_w(w, x, z)


# Teil A – Ein einzelner Update-Schritt (wie Folie Schritt 1–4)

Wir rechnen explizit:
- $\hat z$
- Loss
- Gradienten
- Update


In [ ]:
w = w0.copy()
z_hat = forward(w, x)
err = z_hat - z_target
L = loss(z_hat, z_target)
g = grad_w(w, x, z_target)
w_new = step(w, x, z_target, eta)

print("Gegeben:")
print("  w =", w)
print("  x =", x)
print("  z_target =", z_target)

print("\nVorwärtsrechnung:")
print("  z_hat = sum(w_i * x_i) =", z_hat)
print("  Fehler (z_hat - z) =", err)
print("  Loss = 1/2*(z_hat - z)^2 =", L)

print("\nGradienten:")
print("  dL/dw = (z_hat - z) * x")
print("  dL/dw =", g)

print("\nUpdate:")
print("  w_new = w - eta * dL/dw")
print("  w_new =", w_new)

df_one = pd.DataFrame({
    "x_i": x,
    "w_i": w,
    "Gradient dL/dw_i": g,
    "w_i neu": w_new
}, index=["w11","w22","w33","w44"])

df_one


# Teil B – Mehrere Iterationen (wie die Tabelle in der Folie)

Wir erzeugen die Iterationstabelle:
- Output $\hat z$
- Fehler $(\hat z - z)$
- $w_{11}$, $w_{22}$, $w_{33}$, $w_{44}$


In [ ]:
def run(n_steps: int, w_start: np.ndarray, x: np.ndarray, z: float, eta: float) -> pd.DataFrame:
    w = w_start.copy()
    rows = []
    for it in range(n_steps + 1):
        z_hat = forward(w, x)
        err = z_hat - z
        rows.append([it, z_hat, err, w[0], w[1], w[2], w[3]])
        w = step(w, x, z, eta)
    return pd.DataFrame(rows, columns=["Iteration", "Output z_hat", "Fehler (z_hat-z)", "w11", "w22", "w33", "w44"])

df = run(n_steps=5, w_start=w0, x=x, z=z_target, eta=eta)
df


## Vergleich mit der Folien-Tabelle

In deiner Folie stimmt alles bis Iteration 4.

In Iteration 5 ist in der Folie **$w_{22}$ = +0.15** angegeben.
Mit der mathematisch konsistenten Update-Regel ergibt sich dort **$w_{22}$ ≈ -0.155**.

Das Notebook zeigt beides, damit du die Folie ggf. korrigieren kannst.

In [ ]:
# Folienwerte (abgelesen) – Achtung: vermutlich Vorzeichenfehler in Zeile 5 bei w22
slide_rows = [
    [0, 4.00, 3.50, 1.000, 1.000, 1.000, 1.00],
    [1, 1.90, 1.40, 0.650, 0.300, 0.650, 1.00],
    [2, 1.060, 0.56, 0.510, 0.020, 0.510, 1.00],
    [3, 0.724, 0.224, 0.454, -0.092, 0.454, 1.00],
    [4, 0.590, 0.090, 0.432, -0.137, 0.432, 1.00],
    [5, 0.54, 0.04, 0.42, 0.15, 0.42, 1.00]
]
df_slide = pd.DataFrame(slide_rows, columns=["Iteration", "Output z_hat", "Fehler (z_hat-z)", "w11", "w22", "w33", "w44"])

print("Berechnet (Notebook):")
display(df.round(3))

print("\nFolie (abgelesen):")
display(df_slide)

print("\nDifferenz (Notebook - Folie):")
diff = df.round(3).copy()
for col in diff.columns:
    diff[col] = df.round(3)[col].values - df_slide[col].values
display(diff)


# Zusatz: Visualisierung (Konvergenz)

Wir plotten Output und Fehler über die Iterationen.

In [ ]:
plt.figure(figsize=(7, 3.5))
plt.plot(df["Iteration"], df["Output z_hat"], marker="o")
plt.axhline(z_target, linestyle="--")
plt.title("Konvergenz von z_hat Richtung Ziel z=0.5")
plt.xlabel("Iteration")
plt.ylabel("Output z_hat")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.show()

plt.figure(figsize=(7, 3.5))
plt.plot(df["Iteration"], df["Fehler (z_hat-z)"], marker="o")
plt.axhline(0.0, linestyle="--")
plt.title("Fehler (z_hat - z) über Iterationen")
plt.xlabel("Iteration")
plt.ylabel("Fehler")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.show()


## Beobachtungen (wie auf der Folie)

1. **Stabile Konvergenz**: $\hat z$ nähert sich dem Ziel 0.5.
2. **w22 wird negativ**: Weil $x_2 = 2$ den stärksten Einfluss hat und reduziert werden muss.
3. **w44 bleibt unverändert**: Weil $x_4 = 0$ ⇒ Gradient = 0.
4. **Symmetrie**: w11 und w33 entwickeln sich identisch (weil $x_1 = x_3 = 1$).
